<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/21pipelines_onehot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
Rafa Caballero



# Pipelines

Empezamos con un ejemplo, son datos de clientes de bancos y si han abandonado el banco (churn) tras un número de meses o no.

Los datos están tomados de [aquí](https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/BankPrediction.csv"
df = pd.read_csv(url)
df

### 1 Limpieza y análisis exploratorio

**Ejercicio** ¿Cuántos valores diferentes toma customer_id? ¿qué te sugiere eso?

In [ ]:
# no nos permiten usar datos personales
df2 = df.drop(columns=["customer_id"])
df2

In [ ]:
df2.info()

Vemos que hay dos `object` que normalmente corresponden a  tipo string. Veamos primero `gender`

In [ ]:
df2.gender.value_counts()

como solo son 2 valores podemos dejarlo en una sola columna con valores 0,1:

In [ ]:
df2["genderb"] = 1
df2.loc[df2.gender=="Male", "genderb"] = 0
df2

In [ ]:
df3 = df2.copy()
df3 = df2.drop(columns=["gender"])

En el caso de la columna de los países lo mejor es aplicar one-hot encoding. Una forma sencilla es utilizar el método `get_dummies`

In [ ]:
df4 = pd.get_dummies(df3)
df4

Es cómo mirar los datos numéricos por separado, por eso usamos `df2`

In [ ]:
df2.describe()

In [ ]:
import seaborn as sns

correlaciones = df2.corr(numeric_only = True)
sns.clustermap(correlaciones,
                   method = 'complete',
                   cmap   = 'RdBu',
                   annot  = True,
                   annot_kws = {'size': 8})
plt.show()

Podríamos haber utilizado el dataframe tras el one-hot encoding, pero normalmente no aporta mucha información

In [ ]:
correlaciones = df4.corr()
sns.clustermap(correlaciones,
                   method = 'complete',
                   cmap   = 'RdBu',
                   annot  = True,
                   annot_kws = {'size': 8})
plt.show()

Histogramas y boxplots

In [ ]:
for c in df2.columns:
    fig, [ax1,ax2] = plt.subplots(1,2,figsize=(10, 5))
    df2[c].hist(ax=ax1)
    if np.issubdtype(df2[c].dtype, np.number):
        df2.boxplot(column=c,ax=ax2)
    plt.show()

## Primer modelo

Parece que nos puede interesar escalar y también hacer oversampling o similar. Primero lo vamos a hacer sin nada de esto para obtener una estimación inicial

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 1 dividir columnas
yColumn = "churn"
XColumns = [c for c in df4.columns if c!=yColumn]
X = df4[XColumns]
y = df4[yColumn]

# 2 preparar train y test
test = 0.4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# 3 método y entrenamiento
metodo = LogisticRegression()
modelo = metodo.fit(X_train,y_train)

# 4 evaluar
y_pred = modelo.predict(X_test)
k =  cohen_kappa_score(y_test,y_pred)
print("kappa ",k)
cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=modelo.classes_)
disp.plot()

plt.show()

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

El resultado es prácticamente nulo, parece que apenas hay información útil... Vamos a añadir primero el escalado ¿dónde ponerlo?
El sitio es entre el paso 2 y el 3

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# 1
# ....

# 2
# ...

# y ahora escalamos, solo podemos usar los datos de x_train para "entrenar"
# OJO: solo se escalan las X, no las ys
escalador = StandardScaler()
escalador.fit(X_train)
X_traine = escalador.transform(X_train)
X_teste = escalador.transform(X_test)

# 3 método y entrenamiento
metodo = LogisticRegression()
modelo = metodo.fit(X_traine,y_train)

# 4 evaluar
y_pred = modelo.predict(X_teste)
k =  cohen_kappa_score(y_test,y_pred)
print("kappa ",k)
cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=modelo.classes_)
disp.plot()

plt.show()

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Ideas importantes:

- solo se escalan las X, nunca las y
- Se aprende con x_train, pero el escalador debe afectar tanto a X_train como a X_test

Pegas: ¿cómo hacerlo en el caso de cross validation? ¿Cómo añadir el oversampler?

In [ ]:
from sklearn.pipeline import Pipeline


# 1 dividir columnas
yColumn = "churn"
XColumns = [c for c in df4.columns if c!=yColumn]
X = df4[XColumns]
y = df4[yColumn]

# 2 preparar train y test
test = 0.4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# 3 método y entrenamiento
steps = [('scaler', StandardScaler()), ('Logistic', LogisticRegression())]
metodo = Pipeline(steps)
modelo = metodo.fit(X_train,y_train)

# 4 evaluar
y_pred = modelo.predict(X_test)
k =  cohen_kappa_score(y_test,y_pred)
print("kappa ",k)
cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=modelo.classes_)
disp.plot()

plt.show()

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

La idea es que el escalador queda "pegado" al método, de forma forma solo se hace fit del train y se aplica el escalador entrenado al test.

Un detalle técnico: al utilizar SMOTE, RandomOverSampler, etc. debemos usar la clase Pipeline de la clase imblearn

**Ejercicio** Añadir SMOTE(), RandomOverSampler(), RandomUnderSampler()....lo que se quiera en el pipeline

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression

steps = [('scaler', StandardScaler()),  ('Logistic', LogisticRegression())]
metodo = Pipeline(steps)
modelo = metodo.fit(X_train,y_train)

# 4 evaluar
y_pred = modelo.predict(X_test)
k =  cohen_kappa_score(y_test,y_pred)
print("kappa ",k)
cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=modelo.classes_)
disp.plot()

plt.show()

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Buscando el mejor punto de corte

In [ ]:
# 4 evaluar
y_probs = modelo.predict_proba(X_test)


In [ ]:
# 4 evaluar
xs=[]
ys=[]
for cut in [0.1,0.2,0.3,0.4,0.425,0.45,0.475,0.5,0.525,0.55,0.575,0.6,0.7,0.8,0.9]:
    y_pred = [1 if p[1]>cut else 0 for p in y_probs]
    k =  cohen_kappa_score(y_test,y_pred)
    print(cut,k)
    xs.append(cut)
    ys.append(k)

In [ ]:
plt.plot(xs,ys)
plt.show()

In [ ]:
cut = 0.3
y_pred = [1 if p[1]>cut else 0 for p in y_probs]
k =  cohen_kappa_score(y_test,y_pred)
print("kappa ",k)
cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=modelo.classes_)
disp.plot()

plt.show()

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

El método también vale para validación cruzada

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import RepeatedStratifiedKFold,cross_val_score

steps = [('smote',SMOTE()), ('scaler', StandardScaler()),  ('Logistic', LogisticRegression())]
scorer = make_scorer(cohen_kappa_score)
pipeline = Pipeline(steps=steps)
cv = RepeatedStratifiedKFold(n_splits=20, n_repeats=5)
scores = cross_val_score(pipeline, X, y, scoring=scorer, cv=cv)
scores.mean()